In [ ]:
import copy
import biom
import numpy as np
import pandas as pd
import warnings
from typing import Callable
from scipy.spatial import distance
from skbio import (OrdinationResults,
                   DistanceMatrix)
from scipy.sparse.linalg import svds
from gemelli.optspace import svd_sort
from gemelli.ctf import ctf_table_processing
from gemelli.preprocessing import (build_sparse,
                                   matrix_rclr)
from gemelli._defaults import (DEFAULT_COMP, DEFAULT_MSC,
                               DEFAULT_MFC, DEFAULT_MFF,
                               DEFAULT_TEMPTED_PC,
                               DEFAULT_TEMPTED_EP,
                               DEFAULT_TEMPTED_SMTH,
                               DEFAULT_TEMPTED_RES,
                               DEFAULT_TEMPTED_MAXITER,
                               DEFAULT_TEMPTED_RH as DEFAULT_TRH,
                               DEFAULT_TEMPTED_RHC as DEFAULT_RC,
                               DEFAULT_TEMPTED_SVDC,
                               DEFAULT_TEMPTED_SVDCN as DEFAULT_TSCN)

from gemelli.tempted import (freg_rkhs, bernoulli_kernel)

In [ ]:
def format_time():
    '''
    Normalize time points to be in the same format and keep
    only the defined interval (if defined)
    '''

In [ ]:
def initialization():
    """
    Initialize the loadings for the joint-CTF model
    """
    pass

In [ ]:
def udpate_tabular():
    '''
    Update the tabular loadings (subjects and features) lodings
    '''
    pass

In [ ]:
def update_lambda():
    '''
    Estimates the singular value using the loadings
    from the most recent iteration
    '''
    pass

In [ ]:
def udpate_residuals():
    '''
    Update the tensor to be factorized by subtracting the 
    approximation the previous iteration
    '''
    pass

In [ ]:
def joint_ctf():
    '''
    Joint decomposition of two or more tensors
    '''
    pass

In [ ]:
def joint_ctf_helper():
    '''
    Joint decomposition of two or more tensors
    '''
    pass